In [ ]:
pip install CMake
pip install dlib
pip install face-recognition

In [1]:
import face_recognition


In [2]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

# from PIL import ImageGrab

path = 'traning'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

#### FOR CAPTURING SCREEN RATHER THAN WEBCAM
# def captureScreen(bbox=(300,300,690+300,530+300)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr

encodeListKnown = findEncodings(images)
print('Encoding Complete')



['Akhil.jpg', 'harshitha.jpg', 'Hemanth Korra.jpg', 'kishore.jpg', 'koushik.jpg', 'Monica Waghray.jpg', 'Navneet.jpg', 'pratik.jpg', 'rohit.jpg', 'saipavan.jpg', 'sairam.jpg']
['Akhil', 'harshitha', 'Hemanth Korra', 'kishore', 'koushik', 'Monica Waghray', 'Navneet', 'pratik', 'rohit', 'saipavan', 'sairam']
Encoding Complete


In [1]:
import cv2
import face_recognition
import numpy as np


cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import urllib.request
import bz2
import os

# Define the URL and the local filename
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
file_path = "shape_predictor_68_face_landmarks.dat.bz2"
extracted_file_path = "shape_predictor_68_face_landmarks.dat"

# Download the file
if not os.path.exists(extracted_file_path):
    print("Downloading the shape_predictor_68_face_landmarks.dat file...")
    urllib.request.urlretrieve(url, file_path)

    # Extract the file
    print("Extracting the file...")
    with bz2.BZ2File(file_path) as file, open(extracted_file_path, 'wb') as output:
        output.write(file.read())

    # Clean up the .bz2 file
    os.remove(file_path)
    print("Model downloaded and extracted.")

# Load the model
import dlib
predictor = dlib.shape_predictor(extracted_file_path)

# Now you can use `predictor` as shown in the previous example


Extracting the file...
Model downloaded and extracted.


In [ ]:
import cv2
import numpy as np

# Load the face and eye classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Parameters for blink detection
EYE_DIM_CHANGE_THRESH = 0.1  # Minimum relative change in eye dimensions to consider a blink
BLINK_DETECT_FRAMES = 3      # Frames required to confirm a blink
BLINK_THRESHOLD = 1          # Number of blinks needed to confirm liveness

blink_counter = 0
blink_detected = 0
live_detected = False

cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        eye_dims = []
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            eye_dims.append((ew, eh))

        if len(eye_dims) >= 2:
            eye_dims = sorted(eye_dims, key=lambda dim: dim[0] * dim[1], reverse=True)
            max_eye_width = eye_dims[0][0]
            max_eye_height = eye_dims[0][1]

            if blink_counter == 0:
                initial_eye_width = max_eye_width
                initial_eye_height = max_eye_height

            # Calculate relative change in eye dimensions
            width_change = abs(max_eye_width - initial_eye_width) / initial_eye_width
            height_change = abs(max_eye_height - initial_eye_height) / initial_eye_height

            # Detect blink based on significant dimension change
            if width_change > EYE_DIM_CHANGE_THRESH or height_change > EYE_DIM_CHANGE_THRESH:
                blink_counter += 1
                if blink_counter >= BLINK_DETECT_FRAMES:
                    blink_detected += 1
                    blink_counter = 0
            else:
                blink_counter = 0

            # Liveness determination
            if blink_detected >= BLINK_THRESHOLD:
                live_detected = True
            else:
                live_detected = False

            cv2.putText(frame, f"Blinks: {blink_detected}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"Liveness: {'Live' if live_detected else 'Fake'}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Liveness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
